In [1]:
import pandas as pd
from pathlib import Path

DATA_ROOT   = Path("Data")
OUT         = DATA_ROOT / "processed" / "sorensen"
MANIFEST_CLEAN = Path("Data/processed/emphysema_all/manifest_emphysema_all.csv")


df = pd.read_csv(MANIFEST_CLEAN)
cols_keep = ["slice_key","subject_id","label_name","label_code","preprocessed_path"]
df = df[cols_keep].copy()
df.head()


,slice_key,subject_id,label_name,label_code,preprocessed_path
0,subject24_top,24,NT,1,Data/processed/sorensen/preprocessed/subject24...
1,subject24_middle,24,NT,1,Data/processed/sorensen/preprocessed/subject24...
2,subject24_bottom,24,NT,1,Data/processed/sorensen/preprocessed/subject24...
3,subject9_top,9,NT,1,Data/processed/sorensen/preprocessed/subject9_...
4,subject9_middle,9,NT,1,Data/processed/sorensen/preprocessed/subject9_...


In [2]:
# 2) Split theo subject (grouped) + stratified theo label
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedGroupKFold

# ---- 0) Cột định danh file/slice dùng để so khớp loại trừ ----
PATH_COL = (
    "image_path" if "image_path" in df.columns
    else ("preprocessed_path" if "preprocessed_path" in df.columns
          else ("slice_key" if "slice_key" in df.columns else None))
)
if PATH_COL is None:
    raise ValueError("Không tìm thấy cột định danh (image_path / preprocessed_path / slice_key) trong df.")

# ---- 1) Thứ tự classes (nếu chưa có) ----
try:
    classes
except NameError:
    classes = ["NT","CLE","PSE","PLE"] if {"NT","CLE","PSE","PLE"}.issubset(set(df["label_name"].unique())) \
              else sorted(df["label_name"].unique().tolist())

# ---- 2) Major label theo subject để stratify ----
subject_stats = df.groupby(["subject_id","label_name"]).size().unstack(fill_value=0)
major_label   = subject_stats.idxmax(axis=1)
sub2label     = major_label.to_dict()

# Các subject có PLE và số slice PLE theo subject
ple_subj         = subject_stats.index[subject_stats.get("PLE", 0) > 0].tolist()
ple_cnt_by_subj  = subject_stats.get("PLE", 0)

# ---- 3) Helpers ----
def safe_n_splits(labels, desired, min_allowed=2):
    """Giảm n_splits không vượt quá số lượng lớp ít nhất."""
    vc = pd.Series(labels).value_counts()
    return max(min_allowed, min(desired, int(vc.min())))

def _move_subject(src, dst, sid):
    rows = src[src["subject_id"]==sid]
    src  = src[src["subject_id"]!=sid].reset_index(drop=True)
    dst  = pd.concat([dst, rows], ignore_index=True)
    return src, dst

def _save_and_report(train_df, val_df, test_df):
    print("Số lượng:", { "train": len(train_df), "val": len(val_df), "test": len(test_df) })
    print("Số subject:", {
        "train": train_df["subject_id"].nunique(),
        "val":   val_df["subject_id"].nunique(),
        "test":  test_df["subject_id"].nunique()
    })
    # Lưu CSV chia tập
    SPLIT_DIR = OUT / "splits"
    SPLIT_DIR.mkdir(parents=True, exist_ok=True)
    train_df.to_csv(SPLIT_DIR / "train.csv", index=False)
    val_df.to_csv(SPLIT_DIR / "val.csv", index=False)
    test_df.to_csv(SPLIT_DIR / "test.csv", index=False)
    # Phân bố lớp
    print("\nPhân bố lớp (train):")
    display(train_df["label_name"].value_counts().reindex(classes, fill_value=0).to_frame("count"))
    print("\nPhân bố lớp (val):")
    display(val_df["label_name"].value_counts().reindex(classes, fill_value=0).to_frame("count"))
    print("\nPhân bố lớp (test):")
    display(test_df["label_name"].value_counts().reindex(classes, fill_value=0).to_frame("count"))

# =========================
# CASE A: >= 3 PLE subjects
# =========================
groups  = df["subject_id"].values
y_major = df["subject_id"].map(sub2label).values

if len(ple_subj) >= 3:
    n_splits1 = safe_n_splits(y_major, desired=5, min_allowed=2)
    sgkf = StratifiedGroupKFold(n_splits=n_splits1, shuffle=True, random_state=42)
    fold_pairs = list(sgkf.split(df, y=y_major, groups=groups))

    test_idx   = fold_pairs[0][1]
    remain_idx = fold_pairs[0][0]
    df_remain  = df.iloc[remain_idx].copy()

    groups_r  = df_remain["subject_id"].values
    y_major_r = df_remain["subject_id"].map(sub2label).values
    n_splits2 = safe_n_splits(y_major_r, desired=4, min_allowed=2)

    sgkf2     = StratifiedGroupKFold(n_splits=n_splits2, shuffle=True, random_state=7)
    val_idx_r, train_idx_r = list(sgkf2.split(df_remain, y=y_major_r, groups=groups_r))[0][1], \
                             list(sgkf2.split(df_remain, y=y_major_r, groups=groups_r))[0][0]

    train_df = df_remain.iloc[train_idx_r].reset_index(drop=True)
    val_df   = df_remain.iloc[val_idx_r].reset_index(drop=True)
    test_df  = df.iloc[test_idx].reset_index(drop=True)

    # Sửa sau chia để đảm bảo mỗi tập có đủ lớp
    subj_cls = df.groupby(["subject_id","label_name"]).size().unstack(fill_value=0)

    # Ưu tiên TEST rồi VAL
    for cls in classes:
        if cls not in test_df["label_name"].unique():
            cands = (subj_cls[subj_cls[cls]>0][cls].sort_values(ascending=False).index.tolist())
            cands = [sid for sid in cands if sid in set(train_df["subject_id"].unique())]
            if cands:
                train_df, test_df = _move_subject(train_df, test_df, cands[0])
    for cls in classes:
        if cls not in val_df["label_name"].unique():
            cands = (subj_cls[subj_cls[cls]>0][cls].sort_values(ascending=False).index.tolist())
            cands = [sid for sid in cands if sid in set(train_df["subject_id"].unique())]
            if cands:
                train_df, val_df = _move_subject(train_df, val_df, cands[0])

    # Loại trùng subject
    s_test = set(test_df["subject_id"].unique())
    s_val  = set(val_df["subject_id"].unique())
    train_df = train_df[~train_df["subject_id"].isin(s_test | s_val)].reset_index(drop=True)
    val_df   = val_df[~val_df["subject_id"].isin(s_test)].reset_index(drop=True)

    print(f"Mode: regular (>=3 PLE subjects). PATH_COL={PATH_COL}")
    _save_and_report(train_df, val_df, test_df)

# =========================
# CASE B: chỉ có 2 PLE subjects
# =========================
else:
    print(f"Mode: SPECIAL (only 2 PLE subjects). PATH_COL={PATH_COL}")

    # Chọn subject PLE có ít slice hơn cho TEST để giữ nhiều PLE cho train/val
    ple_subj_sorted = sorted(ple_subj, key=lambda s: int(ple_cnt_by_subj.loc[s]))
    ple_test_sid, ple_train_sid = ple_subj_sorted[0], ple_subj_sorted[1]

    # 1) TEST: trọn subject ple_test_sid + thêm fold từ phần còn lại
    test_df_ple = df[df["subject_id"]==ple_test_sid]
    df_non_test = df[df["subject_id"]!=ple_test_sid].reset_index(drop=True)

    groups_nt  = df_non_test["subject_id"].values
    y_major_nt = df_non_test["subject_id"].map(sub2label).values
    n_splits_nt = safe_n_splits(y_major_nt, desired=5, min_allowed=2)
    sgkf = StratifiedGroupKFold(n_splits=n_splits_nt, shuffle=True, random_state=42)
    folds = list(sgkf.split(df_non_test, y=y_major_nt, groups=groups_nt))
    extra_test_idx = folds[0][1]

    test_df_rest = df_non_test.iloc[extra_test_idx].reset_index(drop=True)
    test_df      = pd.concat([test_df_ple, test_df_rest], ignore_index=True)

    # 2) Pool còn lại cho TRAIN/VAL (loại bỏ các hàng đã vào TEST theo PATH_COL)
    df_pool = df[~df[PATH_COL].isin(test_df[PATH_COL])].reset_index(drop=True)

    # 3) Với subject PLE còn lại: CHIA THEO SLICE cho VAL (>=1) và giữ LẠI >=1 cho TRAIN nếu có thể
    pool_ple  = df_pool[df_pool["subject_id"]==ple_train_sid].reset_index(drop=True)
    pool_rest = df_pool[df_pool["subject_id"]!=ple_train_sid].reset_index(drop=True)

    # số slice PLE còn lại (chỉ 1 subject)
    n_ple_total = len(pool_ple)
    # chọn số slice cho VAL: tối thiểu 1, tối đa n_ple_total-1 (để còn lại >=1 cho TRAIN nếu n_ple_total>1)
    n_val_ple = 1 if n_ple_total >= 1 else 0
    if n_ple_total > 1:
        n_val_ple = min(max(1, int(round(n_ple_total * 0.15))), n_ple_total - 1)
    # (nếu chỉ có đúng 1 slice thì val = 1, train = 0 – trường hợp bất khả kháng)

    if n_val_ple > 0:
        rng = np.random.RandomState(123)
        val_ple_idx = rng.choice(n_ple_total, size=n_val_ple, replace=False)
        val_ple_df  = pool_ple.iloc[val_ple_idx]
        train_ple_df= pool_ple.drop(pool_ple.index[val_ple_idx])
    else:
        val_ple_df  = pool_ple.iloc[:0]
        train_ple_df= pool_ple.copy()

    # 4) Non-PLE: chia theo subject cho TRAIN/VAL
    groups_r  = pool_rest["subject_id"].values
    y_major_r = pool_rest["subject_id"].map(sub2label).values
    n_splits2 = safe_n_splits(y_major_r, desired=4, min_allowed=2)
    sgkf2     = StratifiedGroupKFold(n_splits=n_splits2, shuffle=True, random_state=7)
    _split    = list(sgkf2.split(pool_rest, y=y_major_r, groups=groups_r))[0]
    val_idx_r, train_idx_r = _split[1], _split[0]

    val_rest_df   = pool_rest.iloc[val_idx_r].reset_index(drop=True)
    train_rest_df = pool_rest.iloc[train_idx_r].reset_index(drop=True)

    # GHÉP: cho phép trùng subject ple_train_sid giữa TRAIN và VAL (slice-level split)
    val_df   = pd.concat([val_rest_df,   val_ple_df],   ignore_index=True)
    train_df = pd.concat([train_rest_df, train_ple_df], ignore_index=True)

    # Loại trùng subject với TEST (đảm bảo disjoint theo subject so với TEST)
    s_test = set(test_df["subject_id"].unique())
    train_df = train_df[~train_df["subject_id"].isin(s_test)].reset_index(drop=True)
    val_df   = val_df[~val_df["subject_id"].isin(s_test)].reset_index(drop=True)

    # KHÔNG loại trùng subject giữa TRAIN và VAL đối với ple_train_sid (cho phép overlap slice-level)
    # Nếu có subject trùng khác ngoài ple_train_sid (hiếm), loại khỏi TRAIN để tránh leakage
    overlap = (set(train_df["subject_id"].unique()) & set(val_df["subject_id"].unique())) - {ple_train_sid}
    if overlap:
        train_df = train_df[~train_df["subject_id"].isin(overlap)].reset_index(drop=True)

    # Cảnh báo leakage ở VAL do lấy slice từ subject PLE còn lại
    print(f"WARNING: VAL chứa {(val_df['label_name']=='PLE').sum()} slice PLE từ subject {ple_train_sid}. "
          "Chỉ dùng VAL cho early-stopping; đánh giá tổng quát PLE dựa trên TEST.")

    _save_and_report(train_df, val_df, test_df)


Mode: SPECIAL (only 2 PLE subjects). PATH_COL=preprocessed_path
Số lượng: {'train': 54, 'val': 19, 'test': 42}
Số subject: {'train': 19, 'val': 7, 'test': 14}

Phân bố lớp (train):


,count
label_name,
NT,31
CLE,9
PSE,12
PLE,2



Phân bố lớp (val):


,count
label_name,
NT,3
CLE,3
PSE,12
PLE,1



Phân bố lớp (test):


,count
label_name,
NT,27
CLE,9
PSE,3
PLE,3


In [3]:
# 3) Class weights theo phân bố train
from collections import Counter

classes = ["NT","CLE","PSE","PLE"]  
cnt = Counter(train_df["label_name"])
N = sum(cnt.values()); K = len(classes)
class_weights = {c: (N / (K * cnt.get(c,1))) for c in classes}
class_weights


{'NT': 0.43548387096774194, 'CLE': 1.5, 'PSE': 1.125, 'PLE': 6.75}

In [4]:
# 4) Dataset + Augmentation
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import albumentations as A

label2id = {c:i for i,c in enumerate(classes)}

train_tfms = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05,
                       rotate_limit=8, border_mode=0, value=0, p=0.5),
    A.ElasticTransform(alpha=10, sigma=10*0.05, alpha_affine=5,
                       border_mode=0, value=0, p=0.15),
], p=1.0)

val_tfms = A.Compose([], p=1.0)

class SorensenNPZDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df.reset_index(drop=True)
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        z = np.load(row["preprocessed_path"], allow_pickle=True)
        img = z["img_norm"].astype(np.float32)  # [H,W] in [0,1]
        

        # Albumentations nhận HxWxc -> thêm kênh giả
        img3 = np.expand_dims(img, axis=2)
        if self.transforms is not None:
            img3 = self.transforms(image=img3)["image"]
        img = img3[...,0]  # quay lại HxW

        # To tensor (1,H,W)
        x = torch.from_numpy(img).unsqueeze(0)        # 1 channel
        y = torch.tensor(label2id[row["label_name"]], dtype=torch.long)
        return x, y


In [5]:
# 5) DataLoader
import os
BATCH_SIZE = 8
NUM_WORKERS = 0  # max(0, min(4, os.cpu_count() // 2))

train_ds = SorensenNPZDataset(train_df, transforms=train_tfms)
val_ds   = SorensenNPZDataset(val_df,   transforms=val_tfms)
test_ds  = SorensenNPZDataset(test_df,  transforms=val_tfms)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=NUM_WORKERS, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=True)

# Class weights tensor cho CrossEntropy
weights_tensor = torch.tensor([class_weights[c] for c in classes], dtype=torch.float32)
weights_tensor


tensor([0.4355, 1.5000, 1.1250, 6.7500])

In [6]:
# Model: ResNet18
import torch, torch.nn as nn
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

classes = ["NT","CLE","PSE","PLE"]

def build_resnet18(num_classes=4, in_ch=1):
    model = models.resnet18(
        weights=models.ResNet18_Weights.IMAGENET1K_V1
    )
    # sửa conv1 nhận 1 kênh
    if in_ch == 1:
        old_conv = model.conv1
        model.conv1 = nn.Conv2d(
            in_channels=1,
            out_channels=old_conv.out_channels,
            kernel_size=old_conv.kernel_size,
            stride=old_conv.stride,
            padding=old_conv.padding,
            bias=False,
        )
    # thay FC head
    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features, num_classes)
    return model

model = build_resnet18(num_classes=len(classes), in_ch=1).to(device)

from torchinfo import summary
summary(model, input_size=(1, 1, 512, 512),
        col_names=("input_size","output_size","num_params"),
        depth=3)


cuda


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\QUANG/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████████████████████████████████████████████████████████████████████████| 44.7M/44.7M [07:18<00:00, 107kB/s]


Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
ResNet                                   [1, 1, 512, 512]          [1, 4]                    --
├─Conv2d: 1-1                            [1, 1, 512, 512]          [1, 64, 256, 256]         3,136
├─BatchNorm2d: 1-2                       [1, 64, 256, 256]         [1, 64, 256, 256]         128
├─ReLU: 1-3                              [1, 64, 256, 256]         [1, 64, 256, 256]         --
├─MaxPool2d: 1-4                         [1, 64, 256, 256]         [1, 64, 128, 128]         --
├─Sequential: 1-5                        [1, 64, 128, 128]         [1, 64, 128, 128]         --
│    └─BasicBlock: 2-1                   [1, 64, 128, 128]         [1, 64, 128, 128]         --
│    │    └─Conv2d: 3-1                  [1, 64, 128, 128]         [1, 64, 128, 128]         36,864
│    │    └─BatchNorm2d: 3-2             [1, 64, 128, 128]         [1, 64, 128, 128]         128
│    │    └─ReLU: 3-3     

In [7]:
import torch.optim as optim
import torch.nn.functional as F

# dùng CrossEntropy với class_weights
criterion = nn.CrossEntropyLoss(
    weight=weights_tensor.to(device),
    label_smoothing=0.05
)

optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=15)

BEST_PATH = OUT / "resnet18_best.pth"


In [8]:
import numpy as np
import torch
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

@torch.no_grad()
def evaluate(loader, model, criterion, device, classes):
    model.eval()
    tot_loss, n = 0.0, 0
    probs_all, y_all = [], []

    for x, y in loader:
        x = x.to(device); y = y.to(device)
        logits = model(x)
        loss = criterion(logits, y)
        tot_loss += loss.item() * x.size(0); n += x.size(0)
        probs_all.append(logits.softmax(1).cpu().numpy())
        y_all.append(y.cpu().numpy())

    y_true = np.concatenate(y_all)
    y_prob = np.concatenate(probs_all, axis=0)
    y_pred = y_prob.argmax(1)

    acc = accuracy_score(y_true, y_pred)
    f1  = f1_score(y_true, y_pred, average="macro")

    # AUC OvR theo lớp khả dụng (không NaN)
    per_class_auc, auc_vals = {}, []
    C = len(classes)
    for i, cls in enumerate(classes):
        y_bin = (y_true == i).astype(int)
        if y_bin.min() == y_bin.max():  # thiếu lớp i ở tập này
            per_class_auc[cls] = np.nan
        else:
            auc_i = roc_auc_score(y_bin, y_prob[:, i])
            per_class_auc[cls] = float(auc_i); auc_vals.append(auc_i)
    macro_auc = float(np.mean(auc_vals)) if len(auc_vals) > 0 else np.nan

    return {
        "loss": tot_loss / max(n, 1),
        "acc": acc,
        "f1": f1,
        "auc": macro_auc,
        "per_class_auc": per_class_auc
    }


In [9]:
from tqdm.auto import tqdm
from torch.amp import GradScaler, autocast
from pathlib import Path
import numpy as np
import torch

# Cấu hình
EPOCHS = 30                      
BEST_PATH = OUT / "resnet18_best.pth"
EARLYSTOP_METRIC = "auc"         # "auc" hoặc "f1"
early_patience = 7

# AMP (API mới, sửa cảnh báo)
scaler = GradScaler('cuda' if device.type=='cuda' else 'cpu')

best_score, bad_count = -1.0, 0

for epoch in range(1, EPOCHS+1):
    model.train()
    running, n_seen = 0.0, 0

    pbar = tqdm(total=len(train_loader), desc=f"Epoch {epoch}/{EPOCHS}", leave=False, ncols=100)
    for x, y in train_loader:
        x = x.to(device); y = y.to(device)
        optimizer.zero_grad(set_to_none=True)
        with autocast('cuda' if device.type=='cuda' else 'cpu'):
            logits = model(x)
            loss = criterion(logits, y)
        scaler.scale(loss).backward()
        scaler.step(optimizer); scaler.update()

        running += loss.item() * x.size(0); n_seen += x.size(0)
        pbar.set_postfix_str(f"loss={running/max(n_seen,1):.4f}")
        pbar.update(1)
    pbar.close()

    train_loss = running / max(n_seen, 1)
    val_metrics = evaluate(val_loader, model, criterion, device, classes)
    scheduler.step()

    # log tóm tắt 
    print(f"Epoch {epoch:02d}/{EPOCHS} "
          f"- loss: {train_loss:.4f} "
          f"- val_loss: {val_metrics['loss']:.4f} "
          f"- val_acc: {val_metrics['acc']:.4f} "
          f"- val_f1: {val_metrics['f1']:.4f} "
          f"- val_auc: {val_metrics['auc']:.4f}")

    # Early stopping theo AUC (fallback F1 nếu AUC nan)
    es_metric = (val_metrics["auc"] if (EARLYSTOP_METRIC=="auc" and not np.isnan(val_metrics["auc"]))
                 else val_metrics["f1"])
    if es_metric > best_score + 1e-4:
        best_score = es_metric; bad_count = 0
        torch.save({"model": model.state_dict(), "classes": classes}, BEST_PATH)
        print("  ↑ Saved best:", BEST_PATH.as_posix(), f"({EARLYSTOP_METRIC}={best_score:.4f})")
    else:
        bad_count += 1
        if bad_count >= early_patience:
            print("Early stopping triggered.")
            break

# Đánh giá TEST (an toàn hơn với weights_only)
ckpt = torch.load(BEST_PATH, map_location=device, weights_only=True)
model.load_state_dict(ckpt["model"])
test_metrics = evaluate(test_loader, model, criterion, device, classes)
print("\n=== TEST ===")
print(f"loss={test_metrics['loss']:.4f} | acc={test_metrics['acc']:.3f} | "
      f"macro-F1={test_metrics['f1']:.3f} | macro-AUC={test_metrics['auc']:.3f}")
print("AUC theo lớp:", test_metrics["per_class_auc"])


C:\NCKH\mlenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
                                                                                                    

Epoch 01/30 - loss: 1.4741 - val_loss: 1.4441 - val_acc: 0.1579 - val_f1: 0.0682 - val_auc: 0.4931
  ↑ Saved best: Data/processed/sorensen/cnn_baseline_best.pth (auc=0.4931)


Epoch 02/30 - loss: 1.0812 - val_loss: 1.9084 - val_acc: 0.1579 - val_f1: 0.0682 - val_auc: 0.7339
  ↑ Saved best: Data/processed/sorensen/cnn_baseline_best.pth (auc=0.7339)


Epoch 03/30 - loss: 0.9119 - val_loss: 2.5806 - val_acc: 0.1579 - val_f1: 0.0682 - val_auc: 0.6930


Epoch 04/30 - loss: 0.8884 - val_loss: 2.8811 - val_acc: 0.1579 - val_f1: 0.0682 - val_auc: 0.6833


...

Epoch 09/30 - loss: 0.5951 - val_loss: 2.3011 - val_acc: 0.2105 - val_f1: 0.1964 - val_auc: 0.7145


Epoch 10/30 - loss: 0.6390 - val_loss: 2.0607 - val_acc: 0.2632 - val_f1: 0.2635 - val_auc: 0.7212


Epoch 11/30 - loss: 0.6573 - val_loss: 1.8998 - val_acc: 0.3684 - val_f1: 0.4030 - val_auc: 0.7242


Epoch 12/30 - loss: 0.7000 - val_loss: 1.7131 - val_acc: 0.4737 - val_f1: 0.4426 - val_auc: 0.7336


Epoch 13/30 - loss: 0.6723 - val_loss: 1.5504 - val_acc: 0.5263 - val_f1: 0.5056 - val_auc: 0.7453
Early stopping triggered.

=== TEST ===
loss=1.8401 | acc=0.643 | macro-F1=0.196 | macro-AUC=0.541
AUC theo lớp: {'NT': 0.45432098765432094, 'CLE': 0.7171717171717172, 'PSE': 0.4786324786324786, 'PLE': 0.5128205128205128}
